Imports

In [1]:
import os
import re
from py2neo import Graph

Variablen

In [2]:
# Global variables (a bit messy, might need cleaning up)
home_dir = os.getcwd()
folder_path_words = 'Data\\words'
abs_folder_path_words = os.path.join(home_dir, folder_path_words)

In [3]:
# Connect to the Neo4j database
graph = Graph("bolt://localhost:7687")

Methoden

In [4]:
# Function to process lines of words
def process_line(word1, word2, word3, word4):
    # Initialize an empty result string
    result_string = ""
    
    # Iterate through pairs of words to find shortest paths
    for i, start_word in enumerate([word1, word2, word3, word4]):
        row = ""
        for j, end_word in enumerate([word1, word2, word3, word4]):
            if i == j:
                row += " x "  # Same word, no need to compare
            else:
                # Call the find_shortest_path function
                result = find_shortest_path(start_word, end_word)

                # Check if the result is None
                if result is None:
                    row += " - "
                else:
                    row += f" {result} "

        # Add the row to the result_string
        result_string += f"[{row}] "

    return result_string.strip()
    
    #result = f"Processed: {word1}, {word2}, {word3}, {word4}"
    
    return result

In [5]:
# Get nodes based on their name attributes
def get_node_by_name(name):
    # Clean the string
    name = name.replace("'", '')
    # Create a Cypher query to find a node with the given name attribute
    cypher_query = f"MATCH (n {{name: '{name}'}}) RETURN n LIMIT 1"
    # Run the Cypher query and retrieve the result as a single node
    result = graph.run(cypher_query).evaluate()
    return result

In [6]:
# Find the shortest path between the nodes
def find_shortest_path(start_name, end_name):
    # Check if either start_word or end_word is "-"
    if start_name == "-" or end_name == "-":
        return None
    
    if start_name == end_name:
        return 0
    
    # Get the start and end nodes using the provided names
    start_node = get_node_by_name(start_name)
    end_node = get_node_by_name(end_name)
    
    #print(start_node, end_node)
    #print(f"Node 1: " + start_name)
    #print(f"Node 2: " + end_name)
    
    # Check if either start or end node is not found in the database
    if start_node is None or end_node is None:
        return None
    
    # Create a Cypher query to find the shortest path between the nodes
    cypher_query = (
        f"MATCH "
        "(TRC1:SINGLE_NODE {name: $start_name}), "
        "(TRC2:SINGLE_NODE {name: $end_name}), "
        "p = shortestPath((TRC1)-[*]-(TRC2)) "
        #"WHERE TRC1<>TRC2 "
        "RETURN length(p) AS length"
    )
    
    # Run the Cypher query and retrieve the shortest path length as the result
    result = graph.run(cypher_query, start_name=start_name, end_name=end_name).evaluate()

    return int(result) if result is not None else None


Main

In [7]:
file_name = "words.txt"
file_path = os.path.join(abs_folder_path_words, file_name)


# Read the text document
with open(file_path, 'r') as file:
    lines = file.readlines()

# Process each line
for line in lines:
    # Split the line into words using tab as the delimiter
    words = line.strip().split('\t')

    # Call the function with the words as parameters
    output = process_line(*words)
    print(*words)

    # Print or use the output as needed
    print(output)


# Print the final results
print(f"\n")


ocean Microplastics North American Plastics and pollution
[ x  2  1  - ] [ 2  x  2  - ] [ 1  2  x  - ] [ -  -  -  x ]
amendment Tijuana River North American Salmon
[ x  2  2  2 ] [ 2  x  1  2 ] [ 2  1  x  2 ] [ 2  2  2  x ]
Jack Johnson Marine debris result Deutsche Welle
[ x  2  2  2 ] [ 2  x  1  2 ] [ 2  1  x  1 ] [ 2  2  1  x ]
port oxygen sea water river
[ x  2  1  1 ] [ 2  x  1  1 ] [ 1  1  x  1 ] [ 1  1  1  x ]
dump microplastics description Marine pollution
[ x  2  1  1 ] [ 2  x  2  2 ] [ 1  2  x  2 ] [ 1  2  2  x ]
endowment Australia budget India
[ x  1  1  2 ] [ 1  x  1  1 ] [ 1  1  x  1 ] [ 2  1  1  x ]
ship Marine pollution attempt waste
[ x  1  1  1 ] [ 1  x  2  1 ] [ 1  2  x  1 ] [ 1  1  1  x ]
word - Islands Districts of Indonesia
[ x  -  1  - ] [ -  x  -  - ] [ 1  -  x  - ] [ -  -  -  x ]
discharge lead statute river
[ x  1  1  1 ] [ 1  x  2  1 ] [ 1  2  x  1 ] [ 1  1  1  x ]
ucsb - Chumash people exclusive economic zone
[ x  -  2  2 ] [ -  x  -  - ] [ 2  -  x  2 ] [ 2 